<a href="https://colab.research.google.com/github/EllaN12/20_credit_risk_classification/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!git clone https://github.com/EllaN12/Email-Lead-Scoring-Frontend.git

In [2]:
import sys
sys.path.append('/content/Email-Lead-Scoring-Frontend')

In [3]:
cd '/content/Email-Lead-Scoring-Frontend'

/content/Email-Lead-Scoring-Frontend


In [4]:
pwd

'/content/Email-Lead-Scoring-Frontend'

**Business Problem:**
Marketing Department generates sales by sending promotional emails to subcribers. Though the emails generate sales, but the frequency ticks some members off. As a result, some subscribers unsubscribe, this reduces email growth. In addtion, Revenue has slowed, Email growth has slowed.     


# Cost Calculations

** Understand the Drivers**
 - **Key Insights:**
Company has Large Email List: 100,000.
Email list is growing at 6,000/month less 2500 unsub for total of 3500.
High unsubscribe rates: 500 people per sales email
- **Revenue:**
Company sales cycle is generating about $250,000 per month.
Average Customer Lifetime Value: Estimate $2000/customer.
- **Costs:**
Marketing sends 5 Sales Emails Per Month
5% of lost customers likely to convert if nutured



In [5]:
#!pip install pandas-flavor
import pandas as pd
import numpy as np
#import janitor
import itertools
import plotly.express as px
import pandas_flavor as pf



In [6]:

# Calculating a cost table
def cost_calc_monthly_cost_table(
    email_list_size = 1e5,
    email_list_growth_rate = 0.035,
    sales_emails_per_month = 5,
    unsub_rate_per_sales_email = 0.005,
    customer_conversion_rate = 0.05,
    average_customer_value = 2000,
    n_periods = 12
):
    """

    Args:
        email_list_size ([type], optional): [email list size]. Defaults to 1e5.
        email_list_growth_rate (float, optional): [monthly email list growth rate]. Defaults to 0.035.
        sales_emails_per_month (int, optional): [sales emails per month]. Defaults to 5.
        unsub_rate_per_sales_email (float, optional): [unsubscription rate per email]. Defaults to 0.005.
        customer_conversion_rate (float, optional): [Rate of email subscribers that convert to customers]. Defaults to 0.05.
        average_customer_value (int, optional): [Average Customer Value]. Defaults to 2000.
        n_periods (int, optional): [Number of Months for our cost Table ]. Defaults to 12.

    Returns:
        [type]: [Dataframe: a cost table with the following columns: Period, Email_List_Size_No_Growth, Lost_Customers_No_Growth, Cost_No_Growth, Email_List_Size_Growth, Lost_Customers_Growth, Cost_With_Growth]
    """
    # PERIOD
    period_series = pd.Series(np.arange(0, n_periods), name = "Period")
    cost_table2_df = period_series.to_frame()

    # Email Size - No Growth
    cost_table2_df["Email_List_Size_No_Growth"] = np.repeat(email_list_size, n_periods)

    # Lost Customers - No Growth
    cost_table2_df["Lost_Customers_No_Growth"] = cost_table2_df['Email_List_Size_No_Growth'] * unsub_rate_per_sales_email * sales_emails_per_month * customer_conversion_rate

    # Cost - No Growth
    cost_table2_df["Cost_No_Growth"] = cost_table2_df['Lost_Customers_No_Growth'] * average_customer_value

    # Email List Size - Growth
    cost_table2_df["Email_List_Size_Growth"] = cost_table2_df['Email_List_Size_No_Growth'] * ((1 + email_list_growth_rate)**cost_table2_df['Period'])

    # Lost Customers - Growth
    cost_table2_df["Lost_Customers_Growth"] = cost_table2_df['Email_List_Size_Growth'] * unsub_rate_per_sales_email * sales_emails_per_month * customer_conversion_rate

    # Cost - With Growth
    cost_table2_df["Cost_With_Growth"] = cost_table2_df['Lost_Customers_Growth'] * average_customer_value

    return cost_table2_df

print (cost_calc_monthly_cost_table())





    Period  Email_List_Size_No_Growth  Lost_Customers_No_Growth  \
0        0                   100000.0                     125.0   
1        1                   100000.0                     125.0   
2        2                   100000.0                     125.0   
3        3                   100000.0                     125.0   
4        4                   100000.0                     125.0   
5        5                   100000.0                     125.0   
6        6                   100000.0                     125.0   
7        7                   100000.0                     125.0   
8        8                   100000.0                     125.0   
9        9                   100000.0                     125.0   
10      10                   100000.0                     125.0   
11      11                   100000.0                     125.0   

    Cost_No_Growth  Email_List_Size_Growth  Lost_Customers_Growth  \
0         250000.0           100000.000000             125.

In [7]:
# Estimate the un-subscription rate
def cost_total_unsub_cost(cost_table2_df):
    """Takes input from cost_calc_monthly_cost_table() and produces a summary of the total costs

    Args:
        cost_table2_df ([dataframe]): [output from cost_calc_monthly_cost_table()]

    Returns:
        [dataframe]: [a summarized total cost of email unsubscriptions]
    """
    summary_df =  cost_table2_df[["Cost_No_Growth", "Cost_With_Growth"]] \
        .sum()\
        .to_frame() \
        .transpose()
    return summary_df

print (cost_total_unsub_cost(cost_calc_monthly_cost_table()))

   Cost_No_Growth  Cost_With_Growth
0       3000000.0      3.650490e+06


In [8]:

import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
#import janitor
import itertools
import plotly.express as px
import pandas_flavor as pf

import pandas as pd # Import pandas if not already imported


# Perform Cost simulations around unsubsription rate
def cost_simulate_unsub_cost(
    email_list_monthly_growth_rate = [0,0.035],
    customer_conversion_rate = [0.04, 0.05,0.06],
    **kwargs
):
    """AI is creating summary for cost_simulate_unsub_cost
    Generate a cost simulation for different email list growth rates and customer conversion rates to simulate cost uncertainty.add()

    Args:
        email_list_monthly_growth_rate (list, optional): [list of values for email monthly growth rate to simulate  ]. Defaults to [0,0.035].
        customer_conversion_rate (list, optional): [list of values for customer conversion rate to simulate]. Defaults to [0.04, 0.05,0.06].

    Returns:
        [Dataframe]: [cartesian product of email_list_monthly_growth_rate and customer_conversion_rate with total unsubscribers cost simulation results]
    """

    #Parameter Grid using pandas
    # Create MultiIndex from the product of the input lists
    index = pd.MultiIndex.from_product(
        [email_list_monthly_growth_rate, customer_conversion_rate],
        names=['email_list_monthly_growth_rate', 'customer_conversion_rate']
    )
    # Create a DataFrame from the MultiIndex
    parameter_grid_df = pd.DataFrame(index=index).reset_index()


    # Temporary Function
    # List Comprehension (Simulate Costs)
    def temporary_function(x,y):
        cost_table_df = cost_calc_monthly_cost_table(
            email_list_growth_rate = x,
            customer_conversion_rate = y,
            **kwargs
        )
        cost_summary_df = cost_total_unsub_cost(cost_table_df)
        return cost_summary_df

    # List Comprehension (Simulate Costs)

    summary_list = [temporary_function(x, y) for x, y in zip(parameter_grid_df['email_list_monthly_growth_rate'], parameter_grid_df['customer_conversion_rate'])]

    simulation_results_df = pd.concat(summary_list, axis=0)\
        .reset_index() \
        .drop("index", axis = 1)\
        .merge(parameter_grid_df.reset_index(), left_index = True, right_index = True)

    return simulation_results_df
print (cost_simulate_unsub_cost())

   Cost_No_Growth  Cost_With_Growth  index  email_list_monthly_growth_rate  \
0       2400000.0      2.400000e+06      0                           0.000   
1       3000000.0      3.000000e+06      1                           0.000   
2       3600000.0      3.600000e+06      2                           0.000   
3       2400000.0      2.920392e+06      3                           0.035   
4       3000000.0      3.650490e+06      4                           0.035   
5       3600000.0      4.380588e+06      5                           0.035   

   customer_conversion_rate  
0                      0.04  
1                      0.05  
2                      0.06  
3                      0.04  
4                      0.05  
5                      0.06  


In [9]:
# Plot simulation
def cost_plot_simulated_unsub_costs(simulation_results):
    """AI is creating summary for cost_plot_simulated_unsub_costs:
    # Plot the simulation results

    Args:
        simulation_results ([DataFrame]): [the output from cost_simulate_unsub_cost()]

    Returns:
        [plotly]: [Heatmap of the simulation results]
    """
    simulation_results_wide_df = simulation_results\
        .drop('Cost_No_Growth', axis = 1)\
        .pivot(
            index = 'email_list_monthly_growth_rate',
            columns = 'customer_conversion_rate',
            values = 'Cost_With_Growth'
        )

    fig = px.imshow(
        simulation_results_wide_df,
        origin='lower',
        aspect='auto',
        title='Lead Cost Simulation',
        labels=dict(
            x ='Customer Conversion Rate',
            y ='Monthly Email List Growth Rate',
            color='Cost of Unsubscription'
        )
    )

    return fig

cost_simulate_unsub_cost(
    email_list_monthly_growth_rate=[0.01, 0.02, 0.03],
    customer_conversion_rate=[0.04, 0.05, 0.06],
    email_list_size=100000)\
. pipe(cost_plot_simulated_unsub_costs)



**Conclusion:** 30% of unsubcription rate can potentially cost $4Milion of loss revenue based on the simulation results.

# KPIs Definitionns and Exploratory Data Analysis

In [10]:
!pip install pyjanitor
!pip install configparser
import pandas as pd
import sqlalchemy as sql
import re
import janitor as jn
# Extract and Transform data
def db_read_els_data(conn_string = "sqlite:///00_database/crm_database.sqlite"):
    """Function to read in the Subscribers, Tags, and Transactions tables and
    combine them into a DataFrame with tag_count and made_purchase columns

    Args:
        conn_string (str, optional): Defaults to "sqlite:///00_database/crm_database.sqlite".

    Returns:
        _type_: Pandas DataFrame
    """

    # Connect to engine
    engine = sql.create_engine(conn_string)

    # Raw Data Collect
    with engine.connect() as conn:

        # Subscribers
        subscribers_df = pd.read_sql("SELECT * FROM Subscribers", conn)

        subscribers_df['mailchimp_id'] = subscribers_df['mailchimp_id'].astype('int')

        subscribers_df['member_rating'] = subscribers_df['member_rating'].astype('int')

        subscribers_df['optin_time'] = subscribers_df['optin_time'].astype('datetime64[ns]')

        # Tags
        tags_df = pd.read_sql("SELECT * FROM Tags", conn)

        tags_df['mailchimp_id'] = tags_df['mailchimp_id'].astype("int")

        # Transactions
        transactions_df = pd.read_sql("SELECT * FROM Transactions", conn)

        transactions_df['purchased_at'] = transactions_df['purchased_at'].astype('datetime64[ns]')

        transactions_df['product_id'] = transactions_df['product_id'].astype('int')

    # MERGE TAG COUNTS

    user_events_df = tags_df \
        .groupby('mailchimp_id') \
        .agg(dict(tag = 'count')) \
        .set_axis(['tag_count'], axis=1) \
        .reset_index()

    subscribers_joined_df = subscribers_df \
        .merge(user_events_df, how='left') \
        .fillna(dict(tag_count = 0))

    subscribers_joined_df['tag_count'] = subscribers_joined_df['tag_count'].astype('int')

    # MERGE TARGET VARIABLE
    emails_made_purchase = transactions_df['user_email'].unique()

    subscribers_joined_df['made_purchase'] = subscribers_joined_df['user_email'] \
        .isin(emails_made_purchase) \
        .astype('int')


    return subscribers_joined_df

print(db_read_els_data())

       mailchimp_id          user_full_name                       user_email  \
0                 3       Garrick Langworth      garrick.langworth@gmail.com   
1                 4         Cordell Dickens        cordell.dickens@gmail.com   
2                 8               Inga Dach              inga.dach@gmail.com   
3                10     Ferdinand Bergstrom    ferdinand.bergstrom@gmail.com   
4                11          Justen Simonis         justen.simonis@gmail.com   
...             ...                     ...                              ...   
19914         33405   Ms. Felicity Moore MD   ms.felicity.moore.md@gmail.com   
19915         33406            Shirley Rowe           shirley.rowe@gmail.com   
19916         33407  Jarrett Walker-Carroll  jarrett.walkercarroll@gmail.com   
19917         33408            Tanja Herzog           tanja.herzog@gmail.com   
19918         33409          Farrell Murphy         farrell.murphy@gmail.com   

       member_rating optin_time country

In [11]:
# Read Table Names
def db_read_els_table_names(conn_string = "sqlite:///00_database/crm_database.sqlite"):
    """Reads the Table Names for each table in the crm database.

    Args:
        conn_string (str, optional): Defaults to "sqlite:///00_database/crm_database.sqlite".

    Returns:
        _type_: List with table names
    """

    engine = sql.create_engine(conn_string)

    inspect = sql.inspect(engine)

    table_names = inspect.get_table_names()

    return table_names



/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [12]:
# Get Raw Table
def db_read_raw_els_table(table = "Products", conn_string = "sqlite:///00_database/crm_database.sqlite"):
    """Reads a single raw table from the CRM database.

    Args:
        table (str, optional): Table Name. Defaults to "Products". See
        db_read_els_table_names() to get the full list of table names.
        conn_string (str, optional): Defaults to "sqlite:///00_database/crm_database.sqlite".

    Returns:
        _type_: Pandas DataFrame
    """

    engine = sql.create_engine(conn_string)

    with engine.connect() as conn:

        df = pd.read_sql(
            sql=f"SELECT * FROM {table}",
            con=conn
        )

    return df

print (db_read_raw_els_table())

    product_id                                        description  \
0          1.0  4-Course Bundle - Machine Learning + Expert We...   
1          2.0  4-Course Bundle - Machine Learning + Expert We...   
2          3.0  4-Course Bundle - Machine Learning + Expert We...   
3          4.0  5 Course Bundle - Machine Learning + Web Apps ...   
4          5.0  5 Course Bundle - Machine Learning + Web Apps ...   
5          6.0  5 Course Bundle - Machine Learning + Web Apps ...   
6          7.0  5 Course Bundle - Machine Learning + Web Apps ...   
7          8.0  Bundle - Data Science Starter Kit: R-Track - C...   
8          9.0  Bundle - Data Science Starter Kit: R-Track - C...   
9         10.0  Bundle - DS For Business + Web Apps (Level 1):...   
10        11.0  Bundle - DS For Business + Web Apps (Level 1):...   
11        12.0  Bundle - DS For Business + Web Apps (Level 1):...   
12        13.0  Bundle - DS For Business + Web Apps (Level 1):...   
13        14.0  Bundle - Machine L

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [13]:
!pip install pycaret

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [14]:
# PROCESSING ----
import email_lead_scoring as els
leads_df = els.db_read_els_data()
def process_lead_tags(leads_df, tags_df):
    """Processing Pipeline that combines leads and tags data frames and prepares for machine learning.

    Args:
        leads_df (DataFrame): els.db_read_els_data()
        tags_df (DataFrame): els.db_read_raw_els_table("Tags")

    Returns:
        DataFrame: Leads and Tags combined and prepared for machine learning analysis
    """
    leads_df = els.db_read_els_data()
    # Date Features

    date_max = leads_df['optin_time'].max()

    leads_df['optin_days'] = (leads_df['optin_time'] - date_max).dt.days

    # Email Features

    leads_df['email_provider'] = leads_df['user_email'] \
        .map(lambda x: x.split("@")[1])

    # Activity Features (Rate Features)

    leads_df['tag_count_by_optin_day'] = leads_df['tag_count'] / abs(leads_df['optin_days'] - 1)

    # Specific Tag Features (Actions)

    tags_wide_leads_df = tags_df \
        .assign(value = lambda x: 1) \
        .pivot(
            index = 'mailchimp_id',
            columns = 'tag',
            values = 'value'
        ) \
        .fillna(value = 0) \
        .pipe(
            func=jn.clean_names
        )

    # Merge Tags

    tags_wide_leads_df.columns = tags_wide_leads_df.columns \
        .to_series() \
        .apply(func = lambda x: f"tag_{x}") \
        .to_list()

    tags_wide_leads_df = tags_wide_leads_df.reset_index()

    leads_tags_df = leads_df \
        .merge(tags_wide_leads_df, how='left')

    # Fill NA selectively

    def fillna_regex(data, regex, value = 0, **kwargs):
        for col in data.columns:
            if re.match(pattern=regex, string = col):
                # print(col)
                data[col] = data[col].fillna(value=value, **kwargs)
        return data

    leads_tags_df = fillna_regex(leads_tags_df, regex="^tag_", value = 0)

    # High Cardinality Features: Country Code

    countries_to_keep = [
        'us',
        'in',
        'au',
        'uk',
        'br',
        'ca',
        'de',
        'fr',
        'es',
        'mx',
        'nl',
        'sg',
        'dk',
        'pl',
        'my',
        'ae',
        'co',
        'id',
        'ng',
        'jp',
        'be'
    ]

    leads_tags_df['country_code'] = leads_tags_df['country_code'] \
        .apply(lambda x: x if x in countries_to_keep else 'other')


    return leads_tags_df
print(process_lead_tags(leads_df, tags_df))


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



ModuleNotFoundError: No module named 'mlflow'

In [ ]:
# FINAL PIPELINE ----

def db_read_and_process_els_data(
    conn_string='sqlite:///00_database/crm_database.sqlite'
):
    leads_df = db_read_els_data(conn_string=conn_string)

    tags_df = db_read_raw_els_table(
        table = "Tags",
        conn_string=conn_string
    )

    df = process_lead_tags(leads_df, tags_df)

    return df
print (db_read_and_process_els_data)